# 7 Sensitivity 

This notebook presents computations in section 7 of th paper.

To check the previous section:

[Section 6: A climate component of a social planner's decision problem](sec6_DecisionProblem.ipynb)


texts in section 7


In this section, we focus on brownian misspecification and jump misspecification.
Therefore, in the following analysis and computations, uncertainty parameter for ambiguity aversion, $\xi_a$ is set to $\infty$. 

## 7.1 Robust perspective